In [1]:
import praw
from IPython import display
import pandas as pd
from datetime import datetime

In [2]:
CLIENT_ID='hruo_5_fVGMzQBAg4P_Ysg'
CLIENT_SECRET='j1fE6K4dE6WO2Fb1pv18AwSaVr4Pbg'
USER_AGENT='Ok_Hope_6483'

In [3]:
def get_reddit_headlines(subreddit, query, limit=1000):
    reddit = praw.Reddit(client_id=CLIENT_ID,
                         client_secret=CLIENT_SECRET,
                         user_agent=USER_AGENT)
    
    headlines = []
    for submission in reddit.subreddit(subreddit).search(query, limit=limit):
        # Extracting title and created date
        title = submission.title
        created_utc = datetime.utcfromtimestamp(submission.created_utc)
        headlines.append((title, created_utc))
    
    return headlines

def get_tourism_headlines_for_cities(cities, limit=100):
    headlines_by_city = []
    for city in cities:
        query = f"tourism {city}"
        headlines = get_reddit_headlines('all', query, limit)
        for headline, date in headlines:
            headlines_by_city.append({'City': city, 'Headline': headline, 'Date': date})
    return headlines_by_city

In [4]:
cities = ['Hanoi', 'Saigon', 'Da Nang', 'Sapa', 'Nha Trang', 'Hoi An', 'Hue', 'Phu Quoc', 'Vung Tau', 'Da Lat']
headlines = get_tourism_headlines_for_cities(cities, limit=1000)

In [5]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def analyze_sentiment(headlines):
    analyzer = SentimentIntensityAnalyzer()
    for headline in headlines:
        sentiment_scores = analyzer.polarity_scores(headline['title'])
        headline['sentiment'] = sentiment_scores['compound']
    return headlines

In [6]:
for city in headlines:
    headlines[city] = analyze_sentiment(headlines[city])

TypeError: list indices must be integers or slices, not dict

In [7]:
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(headlines)

# Save the DataFrame to a CSV file
df.to_csv('reddit_tourism_headlines2.csv', index=False)